In [ ]:
!pip install -U groq

In [ ]:
import os

# Prompt the user for the GROQ API Key
api_key = input("Please enter your GROQ API Key: ")
os.environ['GROQ_API_KEY'] = api_key


In [10]:
from groq import Groq
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be attributed to several factors:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in scenarios where timely responses are critical, such as customer support or emergency services.
2. **Low-Latency Requirements**: Many modern applications, like voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications, such as language translation platforms or social media analytics tools.
4. **Energy Efficiency**: Fast lang

In [11]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [12]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this:

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [13]:
import re
import httpx
def wikipedia(q):
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]
#
def calculate(operation: str) -> float:
    return eval(operation)
#

In [14]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break

In [16]:
loop(query="What is current age of Mr. Nadrendra Modi multiplied by 2?")

Thought: I need to find the birthdate of Narendra Modi on Wikipedia.
Action: wikipedia: Narendra Modi
PAUSE
[('wikipedia', 'Narendra Modi')]
Observation: <span class="searchmatch">Narendra</span> Damodardas <span class="searchmatch">Modi</span> (Gujarati: [<span class="searchmatch">ˈnəɾendɾə</span> dɑmodəɾˈdɑs <span class="searchmatch">ˈmodiː</span>] ; born 17 September 1950) is an Indian politician serving as the current prime minister
Thought: I have found the birthdate of Narendra Modi, now I need to calculate his current age and multiply it by 2.
Action: calculate: (2023 - 1950)
PAUSE
[('calculate', '(2023 - 1950)')]
Observation: 73
Thought: I have calculated Narendra Modi's current age, now I need to multiply it by 2.
Action: calculate: 73 * 2
PAUSE
[('calculate', '73 * 2')]
Observation: 146
Thought: I have calculated Narendra Modi's current age multiplied by 2.

Answer: The current age of Mr. Narendra Modi multiplied by 2 is 146.


In [17]:
loop(query="What will be age of Mr. Narendra Modi in 2024 multiplied by 2?")

Thought: I should look up Narendra Modi's age on Wikipedia
Action: wikipedia: Narendra Modi
PAUSE
[('wikipedia', 'Narendra Modi')]
Observation: <span class="searchmatch">Narendra</span> Damodardas <span class="searchmatch">Modi</span> (Gujarati: [<span class="searchmatch">ˈnəɾendɾə</span> dɑmodəɾˈdɑs <span class="searchmatch">ˈmodiː</span>] ; born 17 September 1950) is an Indian politician serving as the current prime minister
Thought: I have found the birthdate of Narendra Modi, now I need to calculate his age in 2024
Action: calculate: 2024 - 1950
PAUSE
[('calculate', '2024 - 1950')]
Observation: 74
Thought: I need to multiply Narendra Modi's age in 2024 by 2
Action: calculate: 74 * 2
PAUSE
[('calculate', '74 * 2')]
Observation: 148
Thought: I have calculated Narendra Modi's age in 2024 multiplied by 2
Answer: Narendra Modi's age in 2024 multiplied by 2 is 148.


In [18]:
loop(query="What is the square root of mass of the earth multiplied by 10?")

Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia: mass of earth
PAUSE
[('wikipedia', 'mass of earth')]
Observation: <span class="searchmatch">Earth</span> <span class="searchmatch">mass</span> (denoted as M🜨, M♁ or ME, where 🜨 and ♁ are the astronomical symbols for <span class="searchmatch">Earth</span>), is a unit <span class="searchmatch">of</span> <span class="searchmatch">mass</span> equal to the <span class="searchmatch">mass</span> <span class="searchmatch">of</span> the planet <span class="searchmatch">Earth</span>
Thought: I didn't quite get the mass of Earth, let me try again
Action: wikipedia: mass of earth in kg
PAUSE
[('wikipedia', 'mass of earth in kg')]
Observation: estimate for the <span class="searchmatch">mass</span> <span class="searchmatch">of</span> <span class="searchmatch">Earth</span> is M🜨 = 5.9722×1024 <span class="searchmatch">kg</span>, with a relative uncertainty <span class="searchmatch">of</span> 10−4. It is equivalent to an average

In [20]:
loop(query="How should I calcualate the number pi")


Thought: I should look up how to calculate pi on Wikipedia.
Action: wikipedia: pi
PAUSE
[('wikipedia', 'pi')]
Observation: The number π (/paɪ/; spelled out as &quot;<span class="searchmatch">pi</span>&quot;) is a mathematical constant that is the ratio of a circle's circumference to its diameter, approximately equal
Thought: I need to find a way to calculate pi, not just read about it. I should look up the formula to calculate pi on Wikipedia.
Action: wikipedia: calculating pi
PAUSE
[('wikipedia', 'calculating pi')]
Observation: January 2020. &quot;<span class="searchmatch">Calculating</span> <span class="searchmatch">Pi</span>: My attempt at breaking the <span class="searchmatch">Pi</span> World Record&quot;. 26 June 2019. Retrieved 30 January 2020. &quot;Die FH Graubünden kennt <span class="searchmatch">Pi</span> am genauesten
Thought: I'm not getting the formula to calculate pi from Wikipedia. I think I need to try a different approach. I've heard of the Bailey-Borwein-Plouffe formu